In [1]:
import numpy as np
from tqdm import tqdm
import pandas as pd
import ast
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('mel_features_dataset.csv')

In [3]:
X = df['Mel_Features'].values
y = df['Ragas'].values

In [ ]:
X_list = np.array(df['Mel_Features'].tolist())
y_list = np.array(df['Ragas'].tolist())

In [4]:
print("Shape of X before reshape:", X.shape)
print("Shape of y before encode:", y.shape)

Shape of X before reshape: (174,)
Shape of y before encode: (174,)


In [5]:
le=LabelEncoder()
ti=le.fit_transform(y)
y_mel=to_categorical(ti)

In [6]:
X_train,X_test,y_train,y_test=train_test_split(X,y_mel,test_size=0.2,random_state=127)

In [7]:
num_classes = len(y_mel[0])
print(num_classes)

9


In [8]:
#X_train_flat = np.array([x.flatten() for x in X_train])
y_train_numerical = np.argmax(y_train, axis=1)
y_test_numerical = np.argmax(y_test, axis=1)
y_train_encoded = to_categorical(y_train_numerical, num_classes=num_classes)
y_test_encoded = to_categorical(y_test_numerical, num_classes=num_classes)

In [9]:
print(X_train[0])

[[-49.23179  -40.900173 -35.22286  ... -80.       -80.       -80.      ]
 [-49.131256 -37.39643  -26.980406 ... -80.       -80.       -80.      ]
 [-50.548172 -39.183334 -25.958849 ... -80.       -80.       -80.      ]
 ...
 [-50.58346  -31.450312 -25.43192  ... -80.       -80.       -80.      ]
 [-50.48502  -32.110584 -25.729435 ... -80.       -80.       -80.      ]
 [-43.868637 -31.801788 -25.15377  ... -80.       -80.       -80.      ]]


In [17]:
print(type(X_train))
print(X_train.shape)
print(type(X_train[0]))

<class 'numpy.ndarray'>
(139,)
<class 'str'>


In [47]:
def remove_nan_from_list_np(data_list):
    temp_array = np.array(data_list)
    return temp_array[~np.isnan(temp_array)].tolist()

In [50]:
import io

def cleaned_string_array(arr, cleaned_list):
  for i, str_array in enumerate(arr):
    try:
      cleaned_str_array = str_array.strip('[]')
      cleaned_str_array = ' '.join(cleaned_str_array.split())
      sio = io.StringIO(cleaned_str_array)
      numerical_array = np.genfromtxt(sio, dtype=float)
      cleaned_list.append(numerical_array)
    except Exception as e:
        print(f"Error processing sample{e}")

In [51]:
loaded_X_train = []
loaded_X_test = []

cleaned_string_array(X_train, loaded_X_train)
cleaned_string_array(X_test, loaded_X_test)

In [52]:
s_Train = pd.Series(loaded_X_train)
s_Test = pd.Series(loaded_X_test)
cleaned_X_train = s_Train.apply(remove_nan_from_list_np)
cleaned_X_test = s_Test.apply(remove_nan_from_list_np)

In [53]:
print(cleaned_X_train.head())

0    [-49.23179, -40.900173, -35.22286, -80.0, -80....
1    [-73.1736, -65.56986, -46.832676, -80.0, -80.0...
2    [-33.193417, -38.61211, -47.274662, -65.94535,...
3    [-26.856775, -29.381496, -30.761852, -69.10777...
4    [-80.0, -80.0, -80.0, -80.0, -80.0, -80.0, -80...
dtype: object


In [55]:
print(cleaned_X_test.shape)

(35,)


In [ ]:
def convert_string_to_list(list_of_arrays, string_list):
  for i, item in enumerate(string_list):
        # Step 1: Evaluate the outermost string literal
        evaluated_list = ast.literal_eval(str(item))
        # Check if the evaluated_list is indeed a list and has at least one element
        if isinstance(evaluated_list, list) and len(evaluated_list) > 0:
            # Get the actual string containing the array data from inside the list
            evaluated_item_str = evaluated_list[0]

        # Step 2: Clean up the evaluated string
        cleaned_string = evaluated_item_str.strip('[]').replace('\n', ' ')
        cleaned_string = re.sub(r'\s+', ' ', cleaned_string).strip()

        if cleaned_string:
            temp_array = np.fromstring(cleaned_string, sep=' ')
            list_of_arrays.append(temp_array)
        else:
            print("Cleaned string was empty. Appending empty NumPy array.")
            list_of_arrays.append(np.array([]))


In [56]:
model_mel = Sequential()
model_mel.add(Conv1D(32, 3, activation='relu', input_shape=(cleaned_X_train.shape[0], 1)))
model_mel.add(MaxPooling1D(pool_size=2, padding='same'))
model_mel.add(Conv1D(64, 3, activation='relu'))
model_mel.add(Flatten())
model_mel.add(Dense(128, activation='relu'))
model_mel.add(Dense(64, activation='relu'))
model_mel.add(Dense(num_classes, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [57]:
model_mel.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model_mel.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 137, 32)        │           128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 69, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 67, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 4288)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │       548,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 9)              │           585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 564,169 (2.15 MB)

 Trainable params: 564,169 (2.15 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
print(X_train.shape)
print(X_test.shape)

(139,)
(35,)


In [58]:
print(y_train_encoded.shape)
print(y_test_encoded.shape)

(139, 9)
(35, 9)


In [59]:
history_mel = model_mel.fit(cleaned_X_train, y_train_encoded, epochs=30, batch_size=32, validation_data=(cleaned_X_test, y_test_encoded))

ValueError: Invalid dtype: object